# Week 03 - AutoFix

This week, you will delve into the practical application of large language models (LLMs) to automate **code remediation**, a key challenge in modern DevOps. You will implement an “AutoFix” system that automatically proposes fixes for issues identified by a language server.

## Goal: Automated Issue Resolution
Your primary task is to implement an `auto_fix` function that serves as the core of this system. This function will act as a bridge between **diagnostic output** and **code changes**, using an LLM to generate a precise solution.

**Input**: An error object returned by the language server (e.g. a type-checking failure).

**Output**: A proposed change in unified diff format to resolve the issue.

Focus on designing an effective LLM prompt and context strategy that reliably converts a structured error message into a correct and syntactically valid unified diff.

## Environment Variables & Imports

Use the following cells to add any needed environment variables (like API keys) **before** loading any of the python modules.

In [ ]:
import initialize_notebook # noqa

In [ ]:
import pathlib
import random
import re

import jinja2

from hslu.dlm03.common import backend as backend_lib
from hslu.dlm03.tools import lint
from hslu.dlm03.util import ipython_utils, unified_diff

## AutoFix Implementation

In [ ]:
BACKEND_CONFIG = backend_lib.Gemini2p5Flash()
BACKEND = BACKEND_CONFIG.get_backend()

In [ ]:
AUTO_FIX_PROMPT_TEMPLATE = """
!!! Write your prompt here !!!
"""
AUTO_FIX_PROMPT = jinja2.Template(AUTO_FIX_PROMPT_TEMPLATE, undefined=jinja2.StrictUndefined)

In [ ]:
def auto_fix(backend: backend_lib.LLMBackend, issue: lint.Issue,
             auto_fix_prompt_template: jinja2.Template,
             retry: int = 5) -> unified_diff.UnifiedDiff:
    # WRITE CODE BELOW
    ...
    # WRITE CODE ABOVE

    response = None
    exceptions = []
    while response is None:
        try:
            response = backend(messages=messages)
        except Exception as e:
            retry -= 1
            exceptions.append(e)
            if retry < 0:
                raise ExceptionGroup("Retry failed.", exceptions)

    choice = random.choice(response.choices)
    diff = choice.message.content
    if code_block := re.match(r"```([ a-z]*)?\n(.*)?```", diff, re.DOTALL):
        diff = code_block.group(2).strip()
    fix_diff = unified_diff.UnifiedDiff.from_string(diff)
    return fix_diff

In [ ]:
LINT_PATH = pathlib.Path("...")
ipython_utils.display_autofix(lambda: lint.lint(LINT_PATH), lambda issue: auto_fix(BACKEND, issue, AUTO_FIX_PROMPT), None,
                              False)